# Lesson 2: Drift Detection using Evidently AI

**Module 6: Monitoring & CI/CD**  
**Estimated Time**: 45 mins  
**Difficulty**: Intermediate

---

## 🎯 Learning Objectives

By the end of this lesson, you will:

✅ Install and use **Evidently AI**, the industry-standard open-source tool for ML monitoring.  
✅ Generate a **Data Drift Report** comparing Training vs Production data.  
✅ Interpret the visualized report (HTML/Dashboard).  
✅ Understand how to integrate this into a pipeline.

---

## 📚 Table of Contents

1. [Why not just manual scripts?](#1-why)
2. [Deep Dive: Evidently AI](#2-evidently)
3. [Hands-On: Generating a Drift Report](#3-hands-on)
4. [Integration with Pipelines](#4-integration)
5. [Interview Preparation](#5-interview)

---

### 🛠️ Setup
We need `evidently` and `sklearn` for sample data.

In [ ]:
pip install evidently scikit-learn pandas -q

## 1. Why not just manual scripts?

In Lesson 1, we wrote 20 lines of code to check for drift in *one* feature.
In production, you have models with **50+ features**. Writing KS-tests for all of them, handling categorical variables, and creating plots is tedious.

**Tools like Evidently AI** automate this:
- Auto-detect categorical vs numerical.
- Select appropriate statistical tests.
- Generate beautiful HTML reports.

## 2. Deep Dive: Evidently AI

Evidently works on the concept of **Reference** vs **Current** datasets.
- **Reference**: The data you trained on (assumed to be "Good").
- **Current**: The new batch of data from production.

It produces **Reports** (Metric visualizations) and **Test Suites** (Pass/Fail checks).

## 3. Hands-On: Generating a Drift Report

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# 1. Load Data
iris = load_iris(as_frame=True)
df = iris.frame

# 2. Simulate Drift
# Reference: First 75 rows
ref_data = df.iloc[:75]
# Current: Last 75 rows (Iris Virginica is different from Setosa!)
curr_data = df.iloc[75:]

# 3. Create Report
report = Report(metrics=[
    DataDriftPreset(), 
])

report.run(reference_data=ref_data, current_data=curr_data)
print("Report Generated!")

In [ ]:
# 4. View Report
# In Jupyter, this renders an interactive dashboard.
report.show_html()

In [ ]:
# 5. Export for Automation (CI/CD)
# We can save it as JSON to parse programmatically
report.save_json("drift_report.json")

import json
with open("drift_report.json") as f:
    result = json.load(f)

# Check if drift detected (True/False)
drift_detected = result['metrics'][0]['result']['dataset_drift']
print(f"Is Dataset Drifted? {drift_detected}")

## 4. Integration with Pipelines

In a real MLOps pipeline (e.g., Airflow or GitHub Actions):

1. **Extract** `current_batch.csv`.
2. **Load** `reference.csv`.
3. **Run** Evidently script.
4. **If Drift Detected** -> Trigger Retraining or Send Slack Alert.
5. **Save** HTML report to S3 for data scientists to debug.

## 5. Interview Preparation

**Q1: How would you automate drift detection?**  
*A1: I would run a daily job (Airflow) that uses a tool like Evidently AI to compare yesterday's inference data against the training set. If the drift score exceeds a threshold (e.g., 0.5), I'd trigger an alert.*

**Q2: Does drift always mean we need to retrain?**  
*A2: No. Sometimes drift is temporary (e.g., Black Friday traffic). Also, if the model performance (accuracy) hasn't dropped, the drift might be on unimportant features.*